## Trabalho 1 - Tecprog
Descrição do trabalho.

A primeira classe foi criada de forma bem simples como é possível visualizar abaixo. Foi utilizado o método de falsificação para ter aprovação do método `generate_xml`

In [1]:
import pytest
import ipytest
from parameterized import parameterized
import unittest
import uuid
ipytest.autoconfig()

## Método de falsificação

In [3]:
%%run_pytest[clean]
class Node:
    def __init__(self, tipo):
        self.tipo = tipo
class DiagramActivity:
    def __init__(self, nome):
        self.nome = nome
        node = Node(0)
        self.nodes = [node]

# Dicionários que retornarão a tag xml correta
TypesOpenDict = {
    0: "<ActivityDiagram name='%s'>"
}
TypesCloseDict = {
    0: "</ActivityDiagram>"
}
# Função principal a ser testada
def generate_xml(Diagram):
    return """<ActivityDiagram name=‘‘nome do diagrama’’>
    </ActivityDiagram>"""

DA = DiagramActivity('nome do diagrama')

def test1_generate_xml():
    assert generate_xml(DA) == """<ActivityDiagram name=‘‘nome do diagrama’’>
    </ActivityDiagram>"""

.                                                                                                                [100%]
1 passed in 0.01s


## Teste parametrizado

In [59]:
%%run_pytest[clean]
class TypeTag:
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = ''
        self.close_tag = ''
        self.terminal = False
    def add_child(self, child):
        self.children.append(child)
    def set_attr(self, attr):
        self.attr = attr
    def get_child_xml(self):
        if(self.terminal):
            return self.open_tag
        else:
            xml = [self.open_tag]
            for child in self.children:
                xml.append(child.get_child_xml())
            xml.append(self.close_tag)
            return '\n'.join(xml)
        
class ActivityDiagram(TypeTag):
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = '<ActivityDiagram %s >'
        self.close_tag = '</ActivityDiagram>'
        self.terminal = False
        
class ActivityDiagramElements(TypeTag):
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = '<ActivityDiagramElements %s >'
        self.close_tag = '</ActivityDiagramElements>'
        self.terminal = False
class StartNode(TypeTag):
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = '<StartNode %s />'
        self.terminal = True
        
class TestSequence(unittest.TestCase):
    AD = ActivityDiagram()
    ADE = ActivityDiagramElements()
    SN = StartNode()
    
    AD.add_child(ADE)
    ADE.add_child(SN)

    @parameterized.expand([
        ["teste 1", SN, '<StartNode %s />'],
        ["teste 2", ADE, "<ActivityDiagramElements %s >\n<StartNode %s />\n</ActivityDiagramElements>"],
        ["teste 3", AD, "<ActivityDiagram %s >\n<ActivityDiagramElements %s >\n<StartNode %s />\n</ActivityDiagramElements>\n</ActivityDiagram>"]])
    
    def test_sequence(self, name, node, expected):
        self.assertEqual(node.get_child_xml(), expected)

...                                                                                                              [100%]
3 passed in 0.01s


## Teste parametrizado: Adicionando atributos as tags

In [72]:
%%run_pytest[clean]
class TypeTag:
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = ''
        self.close_tag = ''
        self.terminal = False
    def add_child(self, child):
        self.children.append(child)
    def set_attr(self, attr):
        self.attr = attr
    def get_child_xml(self):
        attributes = []
        for attr in self.attr:
            attributes.append(attr+"='"+self.attr[attr]+"'")
        if(self.terminal):
            return self.open_tag.replace('%s', ' '.join(attributes))
        else:
            xml = [self.open_tag.replace('%s', ' '.join(attributes))]
            for child in self.children:
                xml.append(child.get_child_xml())
            xml.append(self.close_tag)
            return '\n'.join(xml)
        
class ActivityDiagram(TypeTag):
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = '<ActivityDiagram %s>'
        self.close_tag = '</ActivityDiagram>'
        self.terminal = False
        
class ActivityDiagramElements(TypeTag):
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = '<ActivityDiagramElements %s>'
        self.close_tag = '</ActivityDiagramElements>'
        self.terminal = False
class StartNode(TypeTag):
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = '<StartNode %s/>'
        self.terminal = True
        


class TestSequence(unittest.TestCase):
    AD = ActivityDiagram()
    AD.set_attr({'name': 'nome do diagrama'})
    ADE = ActivityDiagramElements()
    ADE.set_attr({'name': 'nome do bloco de elementos'})
    SN = StartNode()
    
    AD.add_child(ADE)
    ADE.add_child(SN)
    print(AD.get_child_xml())
    @parameterized.expand([
        ["teste 1", SN, '<StartNode />'],
        ["teste 2", ADE, "<ActivityDiagramElements name='nome do bloco de elementos'>\n<StartNode />\n</ActivityDiagramElements>"],
        ["teste 3", AD, "<ActivityDiagram name='nome do diagrama'>\n<ActivityDiagramElements name='nome do bloco de elementos'>\n<StartNode />\n</ActivityDiagramElements>\n</ActivityDiagram>"]])
    def test_sequence(self, name, node, expected):
        self.assertEqual(node.get_child_xml(), expected)

<ActivityDiagram name='nome do diagrama'>
<ActivityDiagramElements name='nome do bloco de elementos'>
<StartNode />
</ActivityDiagramElements>
</ActivityDiagram>
...                                                                                                              [100%]
3 passed in 0.01s


## Teste parametrizado: Adicionando mais tipos de nó

In [79]:
%%run_pytest[clean]
class TypeTag:
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = ''
        self.close_tag = ''
        self.terminal = False
    def add_child(self, child):
        self.children.append(child)
    def add_children(self, children):
        self.children += children
    def set_attr(self, attr):
        self.attr = attr
    def get_child_xml(self):
        attributes = []
        for attr in self.attr:
            attributes.append(attr+"='"+self.attr[attr]+"'")
        if(self.terminal):
            return self.open_tag.replace('%s', ' '.join(attributes))
        else:
            xml = [self.open_tag.replace('%s', ' '.join(attributes))]
            for child in self.children:
                xml.append(child.get_child_xml())
            xml.append(self.close_tag)
            return '\n'.join(xml)
        
class ActivityDiagram(TypeTag):
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = '<ActivityDiagram %s>'
        self.close_tag = '</ActivityDiagram>'
        self.terminal = False
        
class ActivityDiagramElements(TypeTag):
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = '<ActivityDiagramElements %s>'
        self.close_tag = '</ActivityDiagramElements>'
        self.terminal = False
class StartNode(TypeTag):
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = '<StartNode %s/>'
        self.terminal = True
class ActivityNode(TypeTag):
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = '<ActivityNode %s/>'
        self.terminal = True
class DecisionNode(TypeTag):
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = '<DecisionNode %s/>'
        self.terminal = True
class MergeNode(TypeTag):
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = '<MergeNode %s/>'
        self.terminal = True
class FinalNode(TypeTag):
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = '<FinalNode %s/>'
        self.terminal = True     


class TestSequence(unittest.TestCase):
    AD = ActivityDiagram()
    AD.set_attr({'name': 'nome do diagrama'})
    
    ADE = ActivityDiagramElements()
    ADE.set_attr({'name': 'nome do bloco de elementos'})
    
    SN = StartNode()
    SN.set_attr({'name': 'nome do bloco inicial'})
    
    AN = ActivityNode()
    AN.set_attr({'name': 'nome da atividade'})
    
    DN = DecisionNode()
    DN.set_attr({'name': 'nome do nó de decisão'})
    
    MN = MergeNode()
    MN.set_attr({'name': 'nome do nó de fusão'})
    
    FN = FinalNode()
    FN.set_attr({'name': 'nome do nó final'})
    
    AD.add_child(ADE)
    
    ADE.add_children([SN, AN, DN, MN, FN])
    @parameterized.expand([
        ["teste 1", SN, "<StartNode name='nome do bloco inicial'/>"],
        ["teste 2", ADE, "<ActivityDiagramElements name='nome do bloco de elementos'>\n<StartNode name='nome do bloco inicial'/>\n<ActivityNode name='nome da atividade'/>\n<DecisionNode name='nome do nó de decisão'/>\n<MergeNode name='nome do nó de fusão'/>\n<FinalNode name='nome do nó final'/>\n</ActivityDiagramElements>"],
        ["teste 3", AD, "<ActivityDiagram name='nome do diagrama'>\n<ActivityDiagramElements name='nome do bloco de elementos'>\n<StartNode name='nome do bloco inicial'/>\n<ActivityNode name='nome da atividade'/>\n<DecisionNode name='nome do nó de decisão'/>\n<MergeNode name='nome do nó de fusão'/>\n<FinalNode name='nome do nó final'/>\n</ActivityDiagramElements>\n</ActivityDiagram>"]])
    def test_sequence(self, name, node, expected):
        self.assertEqual(node.get_child_xml(), expected)

...                                                                                                              [100%]
3 passed in 0.01s


## Teste parametrizado: Adicionando transitions e id sequencial

In [15]:
%%run_pytest[clean]
class seq:
    def __init__(self):
        self.next = 0
    def next_id(self):
        self.next+=1
        return self.next
sequence = seq()
class TypeTag:
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = ''
        self.close_tag = ''
        self.terminal = False
        self.id = str(sequence.next_id())
    def add_child(self, child):
        self.children.append(child)
    def add_children(self, children):
        self.children += children
    def set_attr(self, attr):
        self.attr = attr
    def get_child_xml(self):
        attributes = []
        if (self.terminal):
            attributes = ['id='+self.id]
        for attr in self.attr:
            attributes.append(attr+"='"+self.attr[attr]+"'")
        if(self.terminal):
            return self.open_tag.replace('%s', ' '.join(attributes))
        else:
            xml = [self.open_tag.replace('%s', ' '.join(attributes))]
            for child in self.children:
                xml.append(child.get_child_xml())
            xml.append(self.close_tag)
            return '\n'.join(xml)
        
class ActivityDiagram(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityDiagram %s>'
        self.close_tag = '</ActivityDiagram>'
        
class ActivityDiagramElements(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityDiagramElements %s>'
        self.close_tag = '</ActivityDiagramElements>'
        
class StartNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<StartNode %s/>'
        self.terminal = True
class ActivityNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityNode %s/>'
        self.terminal = True
class DecisionNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<DecisionNode %s/>'
        self.terminal = True
class MergeNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<MergeNode %s/>'
        self.terminal = True
class FinalNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<FinalNode %s/>'
        self.terminal = True     
class ActivityDiagramTransitions(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityDiagramTransitions %s>'
        self.close_tag = '</ActivityDiagramTransitions>'

class Transition(TypeTag):
    def __init__(self, source, target):
        super().__init__()
        self.open_tag = '<Transition %s/>'
        self.terminal = True
        self.source = source.id
        self.target = target.id
        
class TestSequence(unittest.TestCase):
    # Elementos
    
    AD = ActivityDiagram()
    AD.set_attr({'name': 'nome do diagrama'})
    
    ADE = ActivityDiagramElements()
    ADE.set_attr({'name': 'nome do bloco de elementos'})
    
    SN = StartNode()
    SN.set_attr({'name': 'nome do bloco inicial'})
    
    AN = ActivityNode()
    AN.set_attr({'name': 'nome da atividade'})
    
    DN = DecisionNode()
    DN.set_attr({'name': 'nome do nó de decisão'})
    
    MN = MergeNode()
    MN.set_attr({'name': 'nome do nó de fusão'})
    
    FN = FinalNode()
    FN.set_attr({'name': 'nome do nó final'})
    
    ADE.add_children([SN, AN, DN, MN, FN])
    
    # Transições
    ADT = ActivityDiagramTransitions()
    ADT.set_attr({'name': 'nome do nó de transições'})
    T1 = Transition(SN, AN)
    T2 = Transition(AN, DN)
    T3 = Transition(DN, MN)
    T4 = Transition(MN, FN)
    
    ADT.add_children([T1, T2, T3, T4])
    
    # Diagrama
    AD.add_child(ADE)
    AD.add_child(ADT)
    
    print(AD.get_child_xml())
    @parameterized.expand([
        ["teste 1", SN, "<StartNode id=3 name='nome do bloco inicial'/>"],
        ["teste 2", ADE, "<ActivityDiagramElements name='nome do bloco de elementos'>\n<StartNode id=3 name='nome do bloco inicial'/>\n<ActivityNode id=4 name='nome da atividade'/>\n<DecisionNode id=5 name='nome do nó de decisão'/>\n<MergeNode id=6 name='nome do nó de fusão'/>\n<FinalNode id=7 name='nome do nó final'/>\n</ActivityDiagramElements>"],
        ["teste 3", AD, "<ActivityDiagram name='nome do diagrama'>\n<ActivityDiagramElements name='nome do bloco de elementos'>\n<StartNode id=3 name='nome do bloco inicial'/>\n<ActivityNode id=4 name='nome da atividade'/>\n<DecisionNode id=5 name='nome do nó de decisão'/>\n<MergeNode id=6 name='nome do nó de fusão'/>\n<FinalNode id=7 name='nome do nó final'/>\n</ActivityDiagramElements>\n<ActivityDiagramTransitions name='nome do nó de transições'>\n<Transition id=9/>\n<Transition id=10/>\n<Transition id=11/>\n<Transition id=12/>\n</ActivityDiagramTransitions>\n</ActivityDiagram>"]])
    def test_sequence(self, name, node, expected):
        self.assertEqual(node.get_child_xml(), expected)

<ActivityDiagram name='nome do diagrama'>
<ActivityDiagramElements name='nome do bloco de elementos'>
<StartNode id=3 name='nome do bloco inicial'/>
<ActivityNode id=4 name='nome da atividade'/>
<DecisionNode id=5 name='nome do nó de decisão'/>
<MergeNode id=6 name='nome do nó de fusão'/>
<FinalNode id=7 name='nome do nó final'/>
</ActivityDiagramElements>
<ActivityDiagramTransitions name='nome do nó de transições'>
<Transition id=9/>
<Transition id=10/>
<Transition id=11/>
<Transition id=12/>
</ActivityDiagramTransitions>
</ActivityDiagram>
...                                                                                                              [100%]
3 passed in 0.01s


## Teste parametrizado: Adicionando restrições as transições

In [3]:
%%run_pytest[clean]
class seq:
    def __init__(self):
        self.next = 0
    def next_id(self):
        self.next+=1
        return self.next
sequence = seq()
class TypeTag:
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = ''
        self.close_tag = ''
        self.terminal = False
        self.id = str(sequence.next_id())
    def add_child(self, child):
        self.children.append(child)
    def add_children(self, children):
        self.children += children
    def set_attr(self, attr):
        self.attr = attr
    def get_child_xml(self):
        attributes = []
        if (self.terminal):
            attributes = ['id='+self.id]
        for attr in self.attr:
            attributes.append(attr+"='"+self.attr[attr]+"'")
        if(self.terminal):
            return self.open_tag.replace('%s', ' '.join(attributes))
        else:
            xml = [self.open_tag.replace('%s', ' '.join(attributes))]
            for child in self.children:
                xml.append(child.get_child_xml())
            xml.append(self.close_tag)
            return '\n'.join(xml)

class ActivityDiagram(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityDiagram %s>'
        self.close_tag = '</ActivityDiagram>'
        
        
        
class ActivityDiagramElements(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityDiagramElements %s>'
        self.close_tag = '</ActivityDiagramElements>'
        
class StartNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<StartNode %s/>'
        self.terminal = True
class ActivityNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityNode %s/>'
        self.terminal = True
class DecisionNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<DecisionNode %s/>'
        self.terminal = True
class MergeNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<MergeNode %s/>'
        self.terminal = True
class FinalNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<FinalNode %s/>'
        self.terminal = True     
class ActivityDiagramTransitions(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityDiagramTransitions %s>'
        self.close_tag = '</ActivityDiagramTransitions>'

class Transition(TypeTag):
    def __init__(self, source, target):
        super().__init__()
        self.open_tag = '<Transition %s/>'
        self.terminal = True
        self.source = []
        self.target = []
        self.create_transition(source, target)
    def create_transition(self, source, target):
        if(isinstance(source[0], DecisionNode)):
            if(len(source)==1):
                for t in target:
                    self.target.append(t.id)
            else:
                raise Exception("ActivityDiagramRuleException")
        elif(isinstance(target[0], MergeNode)):
            if(len(target) == 1):
                for s in source:
                    self.source.append(s.id)
            else:
                raise Exception("ActivityDiagramRuleException")
        elif(isinstance(source[0], StartNode)):
            if(len(target)==1):
                self.target = target[0].id
            else:
                raise Exception("ActivityDiagramRuleException")
        elif(isinstance(target[0], FinalNode)):
            if(len(target)==1):
                self.target = target[0].id
            else:
                raise Exception("ActivityDiagramRuleException")
        
class TestSequence(unittest.TestCase):
    # Elementos
    
    AD = ActivityDiagram()
    AD.set_attr({'name': 'nome do diagrama'})
    
    ADE = ActivityDiagramElements()
    ADE.set_attr({'name': 'nome do bloco de elementos'})
    
    SN = StartNode()
    SN.set_attr({'name': 'nome do bloco inicial'})
    
    AN = ActivityNode()
    AN.set_attr({'name': 'nome da atividade'})
    
    DN = DecisionNode()
    DN.set_attr({'name': 'nome do nó de decisão'})
    
    MN = MergeNode()
    MN.set_attr({'name': 'nome do nó de fusão'})
    
    FN = FinalNode()
    FN.set_attr({'name': 'nome do nó final'})
    
    ADE.add_children([SN, AN, DN, MN, FN])
    
    # Transições
    ADT = ActivityDiagramTransitions()
    ADT.set_attr({'name': 'nome do nó de transições'})
    T1 = Transition([SN], [AN])
    T2 = Transition([AN], [DN])
    T3 = Transition([DN], [MN])
    T4 = Transition([MN], [FN])
    
    ADT.add_children([T1, T2, T3, T4])
    
    # Diagrama
    AD.add_child(ADE)
    AD.add_child(ADT)
    
    print(AD.get_child_xml())
    @parameterized.expand([
        ["teste 1", SN, "<StartNode id=3 name='nome do bloco inicial'/>"],
        ["teste 2", ADE, "<ActivityDiagramElements name='nome do bloco de elementos'>\n<StartNode id=3 name='nome do bloco inicial'/>\n<ActivityNode id=4 name='nome da atividade'/>\n<DecisionNode id=5 name='nome do nó de decisão'/>\n<MergeNode id=6 name='nome do nó de fusão'/>\n<FinalNode id=7 name='nome do nó final'/>\n</ActivityDiagramElements>"],
        ["teste 3", AD, "<ActivityDiagram name='nome do diagrama'>\n<ActivityDiagramElements name='nome do bloco de elementos'>\n<StartNode id=3 name='nome do bloco inicial'/>\n<ActivityNode id=4 name='nome da atividade'/>\n<DecisionNode id=5 name='nome do nó de decisão'/>\n<MergeNode id=6 name='nome do nó de fusão'/>\n<FinalNode id=7 name='nome do nó final'/>\n</ActivityDiagramElements>\n<ActivityDiagramTransitions name='nome do nó de transições'>\n<Transition id=9/>\n<Transition id=10/>\n<Transition id=11/>\n<Transition id=12/>\n</ActivityDiagramTransitions>\n</ActivityDiagram>"]])
    def test_get_child_xml(self, name, node, expected):
        self.assertEqual(node.get_child_xml(), expected)
    @parameterized.expand([
        ["teste 4", [DN, SN], [AN], Exception],
        ["teste 5", [AN], [MN, MN], Exception],
        ["teste 6", [SN], [AN, SN], Exception],
        ["teste 7", [AN], [FN, FN], Exception]
    ])
    def test_Transition(self, name, source, target, expected):
        self.assertRaises(expected, Transition, source, target)
        

<ActivityDiagram name='nome do diagrama'>
<ActivityDiagramElements name='nome do bloco de elementos'>
<StartNode id=3 name='nome do bloco inicial'/>
<ActivityNode id=4 name='nome da atividade'/>
<DecisionNode id=5 name='nome do nó de decisão'/>
<MergeNode id=6 name='nome do nó de fusão'/>
<FinalNode id=7 name='nome do nó final'/>
</ActivityDiagramElements>
<ActivityDiagramTransitions name='nome do nó de transições'>
<Transition id=9/>
<Transition id=10/>
<Transition id=11/>
<Transition id=12/>
</ActivityDiagramTransitions>
</ActivityDiagram>
.......                                                                                                          [100%]
7 passed in 0.02s


## Teste parametrizado: Restrições ao adicionar nós iniciais

In [14]:
%%run_pytest[clean]
class seq:
    def __init__(self):
        self.next = 0
    def next_id(self):
        self.next+=1
        return self.next
sequence = seq()
class TypeTag:
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = ''
        self.close_tag = ''
        self.terminal = False
        self.id = str(sequence.next_id())
    def add_child(self, child):
        if(isinstance(child, StartNode)):
            for self_child in self.children:
                    if(isinstance(self_child, StartNode)):
                        raise Exception("ActivityDiagramRuleException")
        self.children.append(child)
    def add_children(self, children):
        for child in children:
            if(isinstance(child, StartNode)):
                for self_child in self.children:
                    if(isinstance(self_child, StartNode)):
                        raise Exception("ActivityDiagramRuleException")
        self.children += children
    def set_attr(self, attr):
        self.attr = attr
    def get_child_xml(self):
        attributes = []
        if (self.terminal):
            attributes = ['id='+self.id]
        for attr in self.attr:
            attributes.append(attr+"='"+self.attr[attr]+"'")
        if(self.terminal):
            return self.open_tag.replace('%s', ' '.join(attributes))
        else:
            xml = [self.open_tag.replace('%s', ' '.join(attributes))]
            for child in self.children:
                xml.append(child.get_child_xml())
            xml.append(self.close_tag)
            return '\n'.join(xml)

class ActivityDiagram(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityDiagram %s>'
        self.close_tag = '</ActivityDiagram>'
        
class ActivityDiagramElements(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityDiagramElements %s>'
        self.close_tag = '</ActivityDiagramElements>'
        
class StartNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<StartNode %s/>'
        self.terminal = True
class ActivityNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityNode %s/>'
        self.terminal = True
class DecisionNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<DecisionNode %s/>'
        self.terminal = True
class MergeNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<MergeNode %s/>'
        self.terminal = True
class FinalNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<FinalNode %s/>'
        self.terminal = True     
class ActivityDiagramTransitions(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityDiagramTransitions %s>'
        self.close_tag = '</ActivityDiagramTransitions>'

class Transition(TypeTag):
    def __init__(self, source, target):
        super().__init__()
        self.open_tag = '<Transition %s/>'
        self.terminal = True
        self.source = []
        self.target = []
        self.create_transition(source, target)
    def create_transition(self, source, target):
        if(isinstance(source[0], DecisionNode)):
            if(len(source)==1):
                for t in target:
                    self.target.append(t.id)
            else:
                raise Exception("ActivityDiagramRuleException")
        elif(isinstance(target[0], MergeNode)):
            if(len(target) == 1):
                for s in source:
                    self.source.append(s.id)
            else:
                raise Exception("ActivityDiagramRuleException")
        elif(isinstance(source[0], StartNode)):
            if(len(target)==1):
                self.target = target[0].id
            else:
                raise Exception("ActivityDiagramRuleException")
        elif(isinstance(target[0], FinalNode)):
            if(len(target)==1):
                self.target = target[0].id
            else:
                raise Exception("ActivityDiagramRuleException")
        
class TestSequence(unittest.TestCase):
    # Elementos
    
    AD = ActivityDiagram()
    AD.set_attr({'name': 'nome do diagrama'})
    
    ADE = ActivityDiagramElements()
    ADE.set_attr({'name': 'nome do bloco de elementos'})
    
    SN = StartNode()
    SN.set_attr({'name': 'nome do bloco inicial'})
    
    AN = ActivityNode()
    AN.set_attr({'name': 'nome da atividade'})
    
    DN = DecisionNode()
    DN.set_attr({'name': 'nome do nó de decisão'})
    
    MN = MergeNode()
    MN.set_attr({'name': 'nome do nó de fusão'})
    
    FN = FinalNode()
    FN.set_attr({'name': 'nome do nó final'})
    
    ADE.add_children([SN, AN, DN, MN, FN])
    
    # Transições
    ADT = ActivityDiagramTransitions()
    ADT.set_attr({'name': 'nome do nó de transições'})
    T1 = Transition([SN], [AN])
    T2 = Transition([AN], [DN])
    T3 = Transition([DN], [MN])
    T4 = Transition([MN], [FN])
    
    ADT.add_children([T1, T2, T3, T4])
    
    # Diagrama
    AD.add_child(ADE)
    AD.add_child(ADT)
    
    print(AD.get_child_xml())
    @parameterized.expand([
        ["teste 1", SN, "<StartNode id=3 name='nome do bloco inicial'/>"],
        ["teste 2", ADE, "<ActivityDiagramElements name='nome do bloco de elementos'>\n<StartNode id=3 name='nome do bloco inicial'/>\n<ActivityNode id=4 name='nome da atividade'/>\n<DecisionNode id=5 name='nome do nó de decisão'/>\n<MergeNode id=6 name='nome do nó de fusão'/>\n<FinalNode id=7 name='nome do nó final'/>\n</ActivityDiagramElements>"],
        ["teste 3", AD, "<ActivityDiagram name='nome do diagrama'>\n<ActivityDiagramElements name='nome do bloco de elementos'>\n<StartNode id=3 name='nome do bloco inicial'/>\n<ActivityNode id=4 name='nome da atividade'/>\n<DecisionNode id=5 name='nome do nó de decisão'/>\n<MergeNode id=6 name='nome do nó de fusão'/>\n<FinalNode id=7 name='nome do nó final'/>\n</ActivityDiagramElements>\n<ActivityDiagramTransitions name='nome do nó de transições'>\n<Transition id=9/>\n<Transition id=10/>\n<Transition id=11/>\n<Transition id=12/>\n</ActivityDiagramTransitions>\n</ActivityDiagram>"]])
    def test_get_child_xml(self, name, node, expected):
        self.assertEqual(node.get_child_xml(), expected)
    @parameterized.expand([
        ["teste 4", [DN, SN], [AN], Exception],
        ["teste 5", [AN], [MN, MN], Exception],
        ["teste 6", [SN], [AN, SN], Exception],
        ["teste 7", [AN], [FN, FN], Exception],
    ])
    def test_Transition(self, name, source, target, expected):
        self.assertRaises(expected, Transition, source, target)
        
    @parameterized.expand([
        ["teste 8", ADE, [SN, SN], Exception]
    ])
    def test_AddChildren(self, name, element, children, expected):
        self.assertRaises(expected, element.add_children, children)
    @parameterized.expand([
        ["teste 9", ADE, SN, Exception]
    ])
    def test_AddChild(self, name, element, child, expected):
        self.assertRaises(expected, element.add_child, child)

<ActivityDiagram name='nome do diagrama'>
<ActivityDiagramElements name='nome do bloco de elementos'>
<StartNode id=3 name='nome do bloco inicial'/>
<ActivityNode id=4 name='nome da atividade'/>
<DecisionNode id=5 name='nome do nó de decisão'/>
<MergeNode id=6 name='nome do nó de fusão'/>
<FinalNode id=7 name='nome do nó final'/>
</ActivityDiagramElements>
<ActivityDiagramTransitions name='nome do nó de transições'>
<Transition id=9/>
<Transition id=10/>
<Transition id=11/>
<Transition id=12/>
</ActivityDiagramTransitions>
</ActivityDiagram>
.........                                                                                                        [100%]
9 passed in 0.02s


## Teste parametrizado: Diagrama de sequência

In [6]:
%%run_pytest[clean]
class seq:
    def __init__(self):
        self.next = 0
    def next_id(self):
        self.next+=1
        return self.next
sequence = seq()
class TypeTag:
    def __init__(self):
        self.children = []
        self.attr = {}
        self.open_tag = ''
        self.close_tag = ''
        self.terminal = False
        self.id = str(sequence.next_id())
    def add_child(self, child):
        if(isinstance(child, StartNode)):
            for self_child in self.children:
                    if(isinstance(self_child, StartNode)):
                        raise Exception("ActivityDiagramRuleException")
        self.children.append(child)
    def add_children(self, children):
        for child in children:
            if(isinstance(child, StartNode)):
                for self_child in self.children:
                    if(isinstance(self_child, StartNode)):
                        raise Exception("ActivityDiagramRuleException")
        self.children += children
    def set_attr(self, attr):
        self.attr = attr
    def get_child_xml(self):
        attributes = []
        if (self.terminal):
            attributes = ['id='+self.id]
        for attr in self.attr:
            attributes.append(attr+"='"+self.attr[attr]+"'")
        if(self.terminal):
            return self.open_tag.replace('%s', ' '.join(attributes))
        else:
            xml = [self.open_tag.replace('%s', ' '.join(attributes))]
            for child in self.children:
                xml.append(child.get_child_xml())
            xml.append(self.close_tag)
            return '\n'.join(xml)

class ActivityDiagram(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityDiagram %s>'
        self.close_tag = '</ActivityDiagram>'
        
class SequenceDiagrams(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<SequenceDiagrams>'
        self.close_tag = '</SequenceDiagrams>'

class Lifelines(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<Lifelines >'
        self.close_tag = '</Lifelines>'

class Lifeline(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<Lifeline %s />'
        self.terminal = True
        
class Fragments(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<Fragments>'
        self.close_tag = '</Fragments>'

class Optional(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<Optional %s/>'
        self.terminal = True

class SequenceDiagram(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<SequenceDiagram %s>'
        self.close_tag = '</SequenceDiagram>'
        
class Message(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<Message %s/>'
        self.terminal = True
class Fragment(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<Fragment %s/>'
        self.terminal = True
        
class ActivityDiagramElements(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityDiagramElements %s>'
        self.close_tag = '</ActivityDiagramElements>'
        
class StartNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<StartNode %s/>'
        self.terminal = True
class ActivityNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityNode %s/>'
        self.terminal = True
class DecisionNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<DecisionNode %s/>'
        self.terminal = True
class MergeNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<MergeNode %s/>'
        self.terminal = True
class FinalNode(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<FinalNode %s/>'
        self.terminal = True     
class ActivityDiagramTransitions(TypeTag):
    def __init__(self):
        super().__init__()
        self.open_tag = '<ActivityDiagramTransitions %s>'
        self.close_tag = '</ActivityDiagramTransitions>'

class Transition(TypeTag):
    def __init__(self, source, target):
        super().__init__()
        self.open_tag = '<Transition %s/>'
        self.terminal = True
        self.source = []
        self.target = []
        self.create_transition(source, target)
    def create_transition(self, source, target):
        if(isinstance(source[0], DecisionNode)):
            if(len(source)==1):
                for t in target:
                    self.target.append(t.id)
            else:
                raise Exception("ActivityDiagramRuleException")
        elif(isinstance(target[0], MergeNode)):
            if(len(target) == 1):
                for s in source:
                    self.source.append(s.id)
            else:
                raise Exception("ActivityDiagramRuleException")
        elif(isinstance(source[0], StartNode)):
            if(len(target)==1):
                self.target = target[0].id
            else:
                raise Exception("ActivityDiagramRuleException")
        elif(isinstance(target[0], FinalNode)):
            if(len(target)==1):
                self.target = target[0].id
            else:
                raise Exception("ActivityDiagramRuleException")
        
class TestSequence(unittest.TestCase):
    # Diagrama de Atividades
    
    AD = ActivityDiagram()
    AD.set_attr({'name': 'nome do diagrama'})
    
    # Elementos
    
    ADE = ActivityDiagramElements()
    ADE.set_attr({'name': 'nome do bloco de elementos'})
    
    SN = StartNode()
    SN.set_attr({'name': 'nome do bloco inicial'})
    
    AN = ActivityNode()
    AN.set_attr({'name': 'nome da atividade'})
    
    DN = DecisionNode()
    DN.set_attr({'name': 'nome do nó de decisão'})
    
    MN = MergeNode()
    MN.set_attr({'name': 'nome do nó de fusão'})
    
    FN = FinalNode()
    FN.set_attr({'name': 'nome do nó final'})
    
    ADE.add_children([SN, AN, DN, MN, FN])
    
    # Transições
    ADT = ActivityDiagramTransitions()
    ADT.set_attr({'name': 'nome do nó de transições'})
    T1 = Transition([SN], [AN])
    T2 = Transition([AN], [DN])
    T3 = Transition([DN], [MN])
    T4 = Transition([MN], [FN])
    
    ADT.add_children([T1, T2, T3, T4])
    
    # Diagrama
    AD.add_child(ADE)
    AD.add_child(ADT)
    
    # Diagrama de Sequência
    SDS = SequenceDiagrams()
    
    LS = Lifelines()
    
    L = Lifeline()
    L.set_attr({'name': 'nome da lifeline'})
    
    FS = Fragments()
    
    O = Optional()
    O.set_attr({'name': 'nome do fragmento', 'representedBy': 'nome do diagrama de sequencia'})
    
    SD = SequenceDiagram()
    SD.set_attr({'name': 'nome do diagrama'})
    
    M = Message()
    M.set_attr({'name': 'nome da mensagem', 'prob': 'valor da probabilidade', 'source': 'nome da lifeline', 'target': 'nome da lifeline'})
    
    F = Fragment()
    F.set_attr({'name': 'nome da mensagem', 'prob': 'valor da probabilidade'})
    
    
    
    LS.add_children([L, L, L])
    FS.add_children([O, O, O])
    SD.add_children([M, M, F, M])
    SDS.add_children([LS, FS, SD, SD])
    
    file_AD = open(AD.attr['name']+'.txt', "w")
    file_AD.write(AD.get_child_xml())
    file_AD.close()
    file_SDS = open('Sequence Diagram.txt', "w")
    file_SDS.write(SDS.get_child_xml())
    file_SDS.close()
    
    @parameterized.expand([
        ["teste 1", SN, "<StartNode id=3 name='nome do bloco inicial'/>"],
        ["teste 2", ADE, "<ActivityDiagramElements name='nome do bloco de elementos'>\n<StartNode id=3 name='nome do bloco inicial'/>\n<ActivityNode id=4 name='nome da atividade'/>\n<DecisionNode id=5 name='nome do nó de decisão'/>\n<MergeNode id=6 name='nome do nó de fusão'/>\n<FinalNode id=7 name='nome do nó final'/>\n</ActivityDiagramElements>"],
        ["teste 3", AD, "<ActivityDiagram name='nome do diagrama'>\n<ActivityDiagramElements name='nome do bloco de elementos'>\n<StartNode id=3 name='nome do bloco inicial'/>\n<ActivityNode id=4 name='nome da atividade'/>\n<DecisionNode id=5 name='nome do nó de decisão'/>\n<MergeNode id=6 name='nome do nó de fusão'/>\n<FinalNode id=7 name='nome do nó final'/>\n</ActivityDiagramElements>\n<ActivityDiagramTransitions name='nome do nó de transições'>\n<Transition id=9/>\n<Transition id=10/>\n<Transition id=11/>\n<Transition id=12/>\n</ActivityDiagramTransitions>\n</ActivityDiagram>"]])
    def test_get_child_xml(self, name, node, expected):
        self.assertEqual(node.get_child_xml(), expected)
    @parameterized.expand([
        ["teste 4", [DN, SN], [AN], Exception],
        ["teste 5", [AN], [MN, MN], Exception],
        ["teste 6", [SN], [AN, SN], Exception],
        ["teste 7", [AN], [FN, FN], Exception],
    ])
    def test_Transition(self, name, source, target, expected):
        self.assertRaises(expected, Transition, source, target)
        
    @parameterized.expand([
        ["teste 8", ADE, [SN, SN], Exception]
    ])
    def test_AddChildren(self, name, element, children, expected):
        self.assertRaises(expected, element.add_children, children)
    @parameterized.expand([
        ["teste 9", ADE, SN, Exception]
    ])
    def test_AddChild(self, name, element, child, expected):
        self.assertRaises(expected, element.add_child, child)
        
    @parameterized.expand([
        ["teste 10", LS, "<Lifelines >\n<Lifeline id=15 name='nome da lifeline' />\n<Lifeline id=15 name='nome da lifeline' />\n<Lifeline id=15 name='nome da lifeline' />\n</Lifelines>"],
        ["teste 11", FS, "<Fragments>\n<Optional id=17 name='nome do fragmento' representedBy='nome do diagrama de sequencia'/>\n<Optional id=17 name='nome do fragmento' representedBy='nome do diagrama de sequencia'/>\n<Optional id=17 name='nome do fragmento' representedBy='nome do diagrama de sequencia'/>\n</Fragments>"],
        ["teste 12", SD, "<SequenceDiagram name='nome do diagrama'>\n<Message id=19 name='nome da mensagem' prob='valor da probabilidade' source='nome da lifeline' target='nome da lifeline'/>\n<Message id=19 name='nome da mensagem' prob='valor da probabilidade' source='nome da lifeline' target='nome da lifeline'/>\n<Fragment id=20 name='nome da mensagem' prob='valor da probabilidade'/>\n<Message id=19 name='nome da mensagem' prob='valor da probabilidade' source='nome da lifeline' target='nome da lifeline'/>\n</SequenceDiagram>"],
        ["teste 13", SDS, "<SequenceDiagrams>\n<Lifelines >\n<Lifeline id=15 name='nome da lifeline' />\n<Lifeline id=15 name='nome da lifeline' />\n<Lifeline id=15 name='nome da lifeline' />\n</Lifelines>\n<Fragments>\n<Optional id=17 name='nome do fragmento' representedBy='nome do diagrama de sequencia'/>\n<Optional id=17 name='nome do fragmento' representedBy='nome do diagrama de sequencia'/>\n<Optional id=17 name='nome do fragmento' representedBy='nome do diagrama de sequencia'/>\n</Fragments>\n<SequenceDiagram name='nome do diagrama'>\n<Message id=19 name='nome da mensagem' prob='valor da probabilidade' source='nome da lifeline' target='nome da lifeline'/>\n<Message id=19 name='nome da mensagem' prob='valor da probabilidade' source='nome da lifeline' target='nome da lifeline'/>\n<Fragment id=20 name='nome da mensagem' prob='valor da probabilidade'/>\n<Message id=19 name='nome da mensagem' prob='valor da probabilidade' source='nome da lifeline' target='nome da lifeline'/>\n</SequenceDiagram>\n<SequenceDiagram name='nome do diagrama'>\n<Message id=19 name='nome da mensagem' prob='valor da probabilidade' source='nome da lifeline' target='nome da lifeline'/>\n<Message id=19 name='nome da mensagem' prob='valor da probabilidade' source='nome da lifeline' target='nome da lifeline'/>\n<Fragment id=20 name='nome da mensagem' prob='valor da probabilidade'/>\n<Message id=19 name='nome da mensagem' prob='valor da probabilidade' source='nome da lifeline' target='nome da lifeline'/>\n</SequenceDiagram>\n</SequenceDiagrams>"]
    ])
    def test_get_child_xml1(self, name, node, expected):
        self.assertEqual(node.get_child_xml(), expected)
    
    

.............                                                                                                    [100%]
13 passed in 0.04s
